In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Import the fastai package

In [2]:
from fastai.imports import *
from fastai.torch_imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [3]:
torch.cuda.set_device(0)

Basic settings, like image direction, image size, basic architecture, and batch size

In [4]:
PATH = "data/monkeys"
sz = 224
arch = resnet34
bs = 58

In [5]:
!ls {PATH}

models	tmp  train  valid


Definition of the data set for the fastai package

In [7]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, sz))

First training of the last layer of the network, 5 epochs, learning rate 0.01

In [8]:
learn = ConvLearner.pretrained(arch, data, precompute=True)
learn.fit(0.01, 5)

100%|██████████| 18/18 [00:21<00:00,  1.17s/it]


epoch      trn_loss   val_loss   accuracy                
    0      1.243169   0.247798   0.963235  
    1      0.606287   0.089737   0.966912                 
    2      0.373507   0.063445   0.977941                 
    3      0.255108   0.057427   0.985294                 
    4      0.185976   0.052528   0.988971                  



[array([0.05253]), 0.9889705882352942]

Pretty good first results!
Next , another round of training, 2 epochs, learning_rate 0.01

In [9]:
learn.precompute = False
learn.fit(0.01,2,cycle_len = 1)

epoch      trn_loss   val_loss   accuracy                   
    0      0.065578   0.051604   0.992647  
    1      0.080943   0.049198   0.988971                   



[array([0.0492]), 0.9889705882352942]

Save already obtained weigths

In [10]:
learn.save("monkey_pre")

In [11]:
learn.load("monkey_pre")

Function to create data

In [12]:
def get_data(sz,bs):
    data = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, sz))
    
    return data if sz>300 else data.resize(340, "tmp")

Increase the size of the images

In [13]:
learn.set_data(get_data(299,bs))

In [14]:
learn.freeze()

2 more epochs

In [15]:
learn.fit(0.01,2,cycle_len = 1)

epoch      trn_loss   val_loss   accuracy                   
    0      0.102757   0.058064   0.988971  
    1      0.097242   0.048754   0.992647                   



[array([0.04875]), 0.9926470588235294]

Unfreeze the layers of resnet, to fine tune

In [16]:
learn.unfreeze()
lr=np.array([1e-4,1e-3,1e-2])

In [ ]:
Final training

In [17]:
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss   accuracy                  
    0      0.154488   0.05342    0.988971  
    1      0.133464   0.035359   0.996324                  
    2      0.103634   0.035892   0.996324                  
    3      0.097235   0.026718   0.996324                   
    4      0.082153   0.022642   0.996324                   
    5      0.083906   0.019701   1.0                        
    6      0.069781   0.021952   1.0                        



[array([0.02195]), 1.0]

In [ ]:
learn.save("monkey_final")